In [173]:
import pandas as pd
import torch
import torch.nn as nn
import plotly.express as px
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils

In [174]:
class PathwayScores(Dataset):

    def __init__(self, csv_file):
        """Initializes instance of class PathwayScores.

        Args:
            csv_file (str): Path to the csv file with the data.

        """
        self.df = pd.read_csv(csv_file, sep='\t')

        # Grouping variable names
        self.features = ['Cell_Cycle', 'HIPPO', 'MYC', 'NOTCH', 'NRF2', 'PI3K', 'TGF.Beta', 'RTK_RAS', 'TP53', 'WNT', 'Hypoxia', 'SRC', 'ESR1', 'ERBB2']
        self.target = "PAM50"
        
        self.df['PAM50'] = self.df['PAM50'].astype('category')
        self.df['PAM50'] = self.df['PAM50'].cat.codes.values

        # Save target and predictors
        self.X = self.df.drop(self.target, axis=1)
        self.y = self.df[self.target]
        
        self.transform = transforms.Compose([transforms.ToTensor()]) 

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if isinstance(idx, torch.Tensor):
            idx = idx.tolist()

        return [self.transform(self.X.iloc[idx].values), self.transform(self.y[idx])]

In [175]:
csv_file = "data/pathwayscores_clinicalinformation_METABRIC.tsv"
dataset = PathwayScores(csv_file)

In [176]:
class Net(nn.Module):

    def __init__(self, D_in, H=20, D_out=1):
        super().__init__()
        self.fc1 = nn.Linear(D_in, H)
        self.fc2 = nn.Linear(H, D_out)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        return x.squeeze()

In [177]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
trainset, testset = random_split(dataset, [train_size, test_size])

trainloader = DataLoader(trainset, batch_size=200, shuffle=True)
testloader = DataLoader(testset, batch_size=200, shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [178]:
D_in, H = 14, 20
net = Net(D_in, H).to(device)

# Loss function
criterion = nn.MSELoss()

# Optimizer
optimizer = optim.Adam(net.parameters(), weight_decay=0.0001)

In [179]:
n_epochs=100

# Train the net
loss_per_iter = []
loss_per_batch = []

for epoch in range(n_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        # Save loss to plot
        running_loss += loss.item()
        loss_per_iter.append(loss.item())

    loss_per_batch.append(running_loss / (i + 1))
    running_loss = 0.0

ValueError: pic should be 2/3 dimensional. Got 1 dimensions.

In [109]:
t_y = torch.tensor(output, dtype=torch.int64)

In [110]:
t_x = torch.tensor(pathways, dtype=torch.float32)

In [111]:
print(t_x.shape)
print(t_y.shape)

torch.Size([1992, 14])
torch.Size([1992])


In [116]:
trainloader = torch.utils.data.DataLoader(data,
  batch_size=20, shuffle=True)

In [115]:
import torch.nn as nn


class Net(nn.Module):

    def __init__(self, D_in, H=15, D_out=1):
        super().__init__()
        self.fc1 = nn.Linear(D_in, H)
        self.fc2 = nn.Linear(H, D_out)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        return x.squeeze()

In [120]:
D_in, H = 14, 10
net = Net(D_in, H)

print(net)
params = list(net.parameters())
loss_fn = nn.MSELoss()

Net(
  (fc1): Linear(in_features=14, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=1, bias=True)
  (relu): ReLU()
)


In [124]:
def train(network, optimizer, loss_fn, num_iters):
  
  loss_history = []
  
  for niter in range(1, num_iters + 1):
    
    optimizer.zero_grad()
    
    # TODO: sample 10 (minibatch size) random samples of dimension expected by NN 
    # https://pytorch.org/docs/stable/torch.html#torch.rand
    x = t_x
    
    # 1) Forward the data through the network
    y_ = network(x)

    # 2) Compute the loss wrt to a zero label
    loss = loss_fn(outputs, t_y)
    
    # 3) Backprop with repsect to the loss function
    loss.backward()
    
    # Store the loss log to plot
    loss_history.append(loss.item())
    
    # 4) Apply the optimizer with a learning step
    optimizer.step()
    
    if niter % 5 == 0:
        print('Step {:2d} loss: {:.3f}'.format(niter, loss_history[-1]))
      
    plt.plot(loss_history)
    plt.xlabel('Niter')
    plt.ylabel('Loss')
    
net = MyNet()
# we will take stochastic gradient descent (SGD) to exemplify the training loop of a neural network
# We first need to handle the parameters that the optimizer will tune, and then we must specify the learning rate (lr) of each
# update step
opt = optim.SGD(net.parameters(), lr=0.01)
train(net, opt, loss_fn, 50)

AttributeError: 'list' object has no attribute 'size'